In [ ]:
# Updated imports for Qiskit 1.0+
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization.applications import Tsp
from qiskit_optimization.algorithms import MinimumEigenOptimizer
import numpy as np

number_of_cities = 4

# Create a random distance matrix with integers between 0 and 100 (symmetric)
# distances[i][j] represents the distance between city i and city j
np.random.seed(42)  # For reproducibility
distances = np.random.randint(1, 101, size=(number_of_cities, number_of_cities))
distances = (distances + distances.T) // 2  # Make symmetric and ensure integers
np.fill_diagonal(distances, 0)  # Distance to same city is 0

tsp = Tsp(distances)
qp = tsp.to_quadratic_program()

# Solve with classical backend
exact_solver = MinimumEigenOptimizer(NumPyMinimumEigensolver())
result = exact_solver.solve(qp)

print("Optimal route:", tsp.interpret(result))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

# Visualize the cities as a graph
graph = nx.from_numpy_array(distances)
pos = nx.spring_layout(graph)
nx.draw(graph, pos, with_labels=True)
labels = nx.get_edge_attributes(graph, 'weight')
nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels)
plt.show()

In [ ]:
from itertools import permutations

# Get the quantum solution route
quantum_route = tsp.interpret(result)

# Calculate quantum solution cost
quantum_cost = sum(distances[quantum_route[i]][quantum_route[(i + 1) % number_of_cities]] 
                  for i in range(number_of_cities))

# Calculate cost for all possible routes
all_routes = list(permutations(range(number_of_cities)))
route_costs = []

for route in all_routes:
    cost = 0
    for i in range(number_of_cities):
        cost += distances[route[i]][route[(i + 1) % number_of_cities]]
    route_costs.append((route, cost))

# Sort routes by cost
route_costs.sort(key=lambda x: x[1])

print("All possible routes and their costs:")
for route, cost in route_costs:
    is_quantum = (list(route) == quantum_route)
    print(f"Route {list(route)}: Cost {cost} {'← Quantum solution' if is_quantum else ''}")

print("\nQuantum solution:")
print(f"Route {quantum_route}: Cost {quantum_cost}")

print("\nBest possible solution:")
print(f"Route {list(route_costs[0][0])}: Cost {route_costs[0][1]}")

if quantum_cost == route_costs[0][1]:
    print("\n✅ Quantum solution found the optimal route!")
else:
    print("\n❌ Quantum solution did not find the optimal route")
    print(f"Difference from optimal: {quantum_cost - route_costs[0][1]}")